# Post 2 - Creating the RFM Table

In this post, I will be creating an RFM table from the data I cleaned and prepared in Post 1

In [6]:
#Loading the required packages
import pandas as pd
import numpy as np
import datetime as dt

Next I will read in the data and see if I have any additional data cleaning to do

In [7]:
#The first step is to read in the data. The data I prepared was written to a csv file - to begin I will read that in.

df = pd.read_csv("Post2_data")

#Checking the top 10 rows of data
df.head(10)

#Looking at the dataframe, I need to delete the very first column.

list(df)

del df['Unnamed: 0']

df.head(10)
#Now I have removed the extra column that shouldn't be there

#I'm going to double check the column types
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354345 entries, 0 to 354344
Data columns (total 9 columns):
InvoiceNo      354345 non-null int64
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null object
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
TotalPrice     354345 non-null float64
dtypes: float64(3), int64(2), object(4)
memory usage: 24.3+ MB


In [8]:
#I need to change the InvoiceDate column to date time for calculating the frequency aspect of the the RFM table
#I will also reset NOW to the most recent date for this too
NOW = dt.datetime(2011,12,10)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354345 entries, 0 to 354344
Data columns (total 9 columns):
InvoiceNo      354345 non-null int64
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
TotalPrice     354345 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 24.3+ MB


Now I am going to create the RFM table. First I will need to aggregate by CustomerID
Recency will be calculated by taking the current time (which I set to 2011,12,10 in the previous code). This will get a measure of how recent the individuals customer was

Frequency will be calculated by taking the lenght or number of invoices the customer had. This count will yield a measure of
how many purchases the customer made or Frequency

Last I will calculate the monetary value of the purchases. By taking the sum of TotalPrice for each purchase, I will get a measure 
how much the customer has spent at the online retailer

In [15]:
#Aggregating on Customer ID and putting the correct columns in
rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days,
                                        'InvoiceNo': lambda x: len(x),  
                                        'TotalPrice': lambda x: x.sum()})


#Next I will need to change the InvoiceDate from datetime to 
rfm['InvoiceDate'] = rfm['InvoiceDate'].astype(int)
rfm.rename(columns={'InvoiceDate': 'Recency', 
                         'InvoiceNo': 'Frequency', 
                         'TotalPrice': 'Monetary_Value'}, inplace=True)

#Now to look at the RFM table
rfm.head(10)

,Recency,Frequency,Monetary_Value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34
12821.0,214,6,92.72
12822.0,70,46,948.88
12823.0,74,5,1759.50
12824.0,59,25,397.12


In [16]:
#Now to write the RFM table to a csv to use in Post 3
rfm.to_csv('Post3_data')

Interpretation of the top 3 rows of the RFM table

Row (1): Customer 12346 has a Recency of 325 - so the last purchase was made nearly a year ago, a single purchase (b/c frequency =1) and a monetary value of $77,183.60.

Row (2): Customer 124747 has a Recency of 2 - so the last purchase was made two days ago, a frequency of 103, so they are making purchases  more frequently than customer 12346
and a monetary value of $4,196 so the overall value of their purchases are less than customer 12346

Row (3): Customer 12748 has a Recency of 0, so their last purchase was made NOW. This customer has made 4,596 purchases - so they are 
making purchases very frequently and the monetary value of the purcchases is $33,719 - so far and away spending the least per purchase.


Now I will perform so rudimentary analysis to see who the most valuable customers are.

In [17]:
#I will start by digging into the first customer - the code below subsets on the first customer
first_customer = df[df['CustomerID']== 12346.0]
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
33413,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


So we can see that this customer has only shopped once, but has made a purchase of 74,215 units at $1.04 per unit yielding
$77,183.60 in revenue. While this customer placed the order nearly a year ago (from 'NOW'), it was for a very large amount. 
Now I'm going to compare this with another customer.

In [18]:
#Digging into the second customer ID
second_customer = df[df['CustomerID']==12747.0]
second_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
6811,537215,85124C,GREEN JUICY FRUIT PHOTO FRAME,12,2010-12-05 15:38:00,2.55,12747.0,United Kingdom,30.60
6812,537215,85124B,BLUE JUICY FRUIT PHOTO FRAME,6,2010-12-05 15:38:00,2.55,12747.0,United Kingdom,15.30
6813,537215,84879,ASSORTED COLOUR BIRD ORNAMENT,16,2010-12-05 15:38:00,1.69,12747.0,United Kingdom,27.04
6814,537215,85062,PEARL CRYSTAL PUMPKIN T-LIGHT HLDR,24,2010-12-05 15:38:00,1.65,12747.0,United Kingdom,39.60
6815,537215,85064,CREAM SWEETHEART LETTER RACK,6,2010-12-05 15:38:00,5.45,12747.0,United Kingdom,32.70
6816,537215,82484,WOOD BLACK BOARD ANT WHITE FINISH,36,2010-12-05 15:38:00,5.55,12747.0,United Kingdom,199.80
6817,537215,21136,PAINTED METAL PEARS ASSORTED,8,2010-12-05 15:38:00,1.69,12747.0,United Kingdom,13.52
16152,538537,22795,SWEETHEART RECIPE BOOK STAND,16,2010-12-13 10:41:00,5.95,12747.0,United Kingdom,95.20
16153,538537,48138,DOORMAT UNION FLAG,2,2010-12-13 10:41:00,7.95,12747.0,United Kingdom,15.90
16154,538537,82494L,WOODEN FRAME ANTIQUE WHITE,24,2010-12-13 10:41:00,2.55,12747.0,United Kingdom,61.20


This customer had a much higher frequency and recency of purchase, but it was for about 6% of the monetary value of the first
customer's single order. However, because we are looking for repeat sales, I will make the criteria for "best customers" have
the lowest recency, highest frequency and highest monetary value. One way of looking at that will be to divide the customers
into quartiles by recency, frequency and monetary value. Then I will add a column for each quartile rank for recency frequency
and monetary value. Then it will be a simple matter of sorting to see which customers are the best.

In [19]:
#Setting up the quantiles
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

#Creating the segmented RFM Table
segmented_rfm = rfm

#'Scoring'
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
    
#Adding the segment numbers to the newly creataed RFM Table   
segmented_rfm['r_quartile'] = segmented_rfm['Recency'].apply(RScore, args=('Recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['Monetary_Value'].apply(FMScore, args=('Monetary_Value',quantiles,))
segmented_rfm.head()

,Recency,Frequency,Monetary_Value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


Basically this scoring plotted the customers in 3-dimensional space. with a score of '1-1-1' being the best and '4-4-4' being
being the worst. Now I will add a new column that contains the aggregate scores

In [20]:
#Creating the aggregate scores column
segmented_rfm['RFM_AggScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str)+ segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,Recency,Frequency,Monetary_Value,r_quartile,f_quartile,m_quartile,RFM_AggScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [21]:
#Now I will sort on RMF_AggScore for the top 10 customers using this analysis. I will use Monetary Value as the tie breaker
segmented_rfm[segmented_rfm['RFM_AggScore']=='111'].sort_values('Monetary_Value', ascending=False).head(10)


,Recency,Frequency,Monetary_Value,r_quartile,f_quartile,m_quartile,RFM_AggScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111


In [25]:
#I will now write this segmented RFM table to a csv to use later
segmented_rfm.to_csv('Segmented_RFM')

Now to make it easy to examine the "Top" Customers, I will now functionalize this code so that it makes it very easy to pull the top customers.This custom function will allow me to input the number of top customers I want to see, so that I could easily look at the top 5, 10, 15, or 200 for example.

In [23]:
#Function
def top_customers(x):
   print(segmented_rfm[segmented_rfm['RFM_AggScore']=='111'].sort_values('Monetary_Value', ascending=False).head(x))
   return;

In [24]:
top_customers(15)

            Recency  Frequency  Monetary_Value  r_quartile  f_quartile  \
CustomerID                                                               
18102.0           0        431       259657.30           1           1   
17450.0           8        337       194550.79           1           1   
17511.0           2        963        91062.38           1           1   
16684.0           4        277        66653.56           1           1   
14096.0           4       5111        65164.79           1           1   
13694.0           3        568        65039.62           1           1   
15311.0           0       2379        60767.90           1           1   
13089.0           2       1818        58825.83           1           1   
15769.0           7        130        56252.72           1           1   
15061.0           3        403        54534.14           1           1   
14298.0           8       1637        51527.30           1           1   
14088.0          10        589        

While this type of analysis is simple and doesn't utilize any high-powered algorithms, I think it has alot of value. It's simplicity allows a non-technical audience to follow along fairly easily and it does answer the question: "Who are our top customers." Now that the rudimentary analysis is complete, I will utilize k means clustering in the next post